In [1]:
# Import Packages:
from __future__ import print_function

import os, sys
from six.moves import cPickle as pickle
from six.moves import range

# Model Packeges import
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix

# Get the current working directory
cwd = os.getcwd()
mydir = os.path.abspath(os.path.join(cwd, ".."))
sys.path.append(mydir)
from DataGenerator import genTrainValidFolds

In [2]:
STDbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/STD/batchData/"
EDGbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/EDG/batchData/"
HOGp1batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp1/batchData/"   # dim =162
HOGp2batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp2/batchData/"   # dim =576
HOGp3batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp3/batchData/"

featureDIR = HOGp3batch_dir

In [3]:
# def toOneHotVector():
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()
    
    
def reshape_data(dataset, labels, featureSize, numLabels, sample_size=None):
    if sample_size:
        dataset = dataset[:sample_size].reshape(sample_size, featureSize) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    else:
        dataset = dataset.reshape(len(dataset), featureSize) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    return dataset, labels


def accuracy(predictions, labels):
    # Both the predictions and the labels should be in One-Hot vector format.
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])


def confusionMatrix(predictions, labels):
    # Both the predictions and the labels should be in One-Hot vector format.
    return (pd.crosstab(np.argmax(labels, 1), np.argmax(predictions, 1), rownames=['True'], colnames=['Predicted'], margins=True))


In [4]:
class GlabalVariables(object):
    
    def __init__(self):
        super(GlabalVariables, self).__init__()

        self.seed = 2316
        self.imageSize = 32
#         self.featureSize = 1098     # Note to change the featureSize when using a new Feature Set
        self.numHidden1 = 1024
        self.numHidden2 = 1024
        self.numHidden3 = 1024
        self.numLabels = 2
        self.alpha = 0.006          # Learning Rate 
#         self.beta = 0.001           # Penalty for regularization
        self.momentum = 0.9        # Momentum for momentum Optimizer
        self.numFolds = 10
        self.dropout = 0.5
        
        # Learning Variables     
        self.epochs = 60
        self.batchSize = 500

In [5]:
class BuildNeuralNet(GlabalVariables):
    
    def __init__(self, featureSize):
        super(BuildNeuralNet, self).__init__()
        
        self.featureSize = featureSize
        self.weights = {
            "inp_to_hid1_wght": tf.Variable(tf.random_normal([self.featureSize, self.numHidden1], seed=self.seed)),
            "hid1_to_hid2_wght" : tf.Variable(tf.random_normal([self.numHidden1, self.numHidden2], seed=self.seed)),
            "hid2_to_hid3_wght": tf.Variable(tf.random_normal([self.numHidden2, self.numHidden3], seed=self.seed)),
            "hid3_to_out_wght": tf.Variable(tf.random_normal([self.numHidden3, self.numLabels], seed=self.seed))
        }
        
        self.biases = {
            "hid1_bias": tf.Variable(tf.random_normal([self.numHidden1], seed=self.seed)),
            "hid2_bias": tf.Variable(tf.random_normal([self.numHidden2], seed=self.seed)),
            "hid3_bias": tf.Variable(tf.random_normal([self.numHidden3], seed=self.seed)),
            "out_bias": tf.Variable(tf.random_normal([self.numLabels], seed=self.seed))
        }
        
        
    def trainGraph(self, regularization=False):
        trainData = tf.placeholder(tf.float32, [None, self.featureSize])
        trainLabels = tf.placeholder(tf.float32, [None, self.numLabels])

        # 1st Hidden Layer State
        inp_to_hid1 = tf.matmul(trainData, self.weights['inp_to_hid1_wght']) + self.biases['hid1_bias']
#         hid1State = tf.sigmoid(inp_to_hid1)
        hid1State = tf.nn.relu(inp_to_hid1)
        hid1State = tf.nn.dropout(hid1State, self.dropout)
        
        # 2nd Hidden Layer State
        hid1_to_hid2 = tf.matmul(hid1State, self.weights['hid1_to_hid2_wght']) + self.biases['hid2_bias']
#         hid2State = tf.sigmoid(hid1_to_hid2)
        hid2State = tf.nn.relu(hid1_to_hid2)
        hid2State = tf.nn.dropout(hid2State, self.dropout)
        
        # 3nd Hidden Layer State
        hid2_to_hid3 = tf.matmul(hid2State, self.weights['hid2_to_hid3_wght']) + self.biases['hid3_bias']
#         hid3State = tf.sigmoid(hid2_to_hid3)
        hid3State = tf.nn.relu(hid2_to_hid3)
#         hid3State = tf.nn.dropout(hid3State, self.dropout)
        
        # Output Layer State
        hid3_to_out = tf.matmul(hid3State, self.weights['hid3_to_out_wght']) + self.biases['out_bias']
        outState = tf.nn.softmax(hid3_to_out)
        
        # Loss Function
        lossCE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hid3_to_out, labels=trainLabels))
        
        # Loss function using L2 Regularization
        if regularization:
            regularizers = (tf.nn.l2_loss(self.weights['inp_to_hid1_wght']) 
                            + tf.nn.l2_loss(self.weights['hid1_to_hid2_wght']) 
                            + tf.nn.l2_loss(self.weights['hid2_to_hid3_wght']) 
                            + tf.nn.l2_loss(self.weights['hid3_to_out_wght'])
                        )
            lossCE = tf.reduce_mean(lossCE + (self.beta * regularizers))
        
        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(lossCE)
#         optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(lossCE)
#         optimizer = tf.train.MomentumOptimizer(self.alpha, 
#                                             self.momentum, 
#                                             use_locking=False, 
#                                             name='Momentum', 
#                                             use_nesterov=True).minimize(lossCE)
#         optimizer = tf.train.AdagradOptimizer(learning_rate=0.04).minimize(lossCE)
#         optimizer = tf.train.RMSPropOptimizer(learning_rate=0.01, momentum=0.9).minimize(lossCE)   # This setting seems to be good
    
    
        
        # Evaluate model
#         trainPred = tf.equal(tf.argmax(outState, 1), tf.argmax(trainLabels, 1))
#         trainAccuracy = tf.reduce_mean(tf.cast(trainPred, tf.float32))
        
        return dict(
            trainData = trainData, 
            trainLabels = trainLabels,
            trainPred = outState,
            optimizer = optimizer,
            lossCE = lossCE
        )
    
    
    def validGraph(self):

        validData = tf.placeholder(tf.float32, [None, self.featureSize])
        validLabels = tf.placeholder(tf.float32, [None, self.numLabels])

        # 1st Hidden Layer State
        valid_hid1State = tf.sigmoid(tf.matmul(validData, self.weights['inp_to_hid1_wght']) + self.biases['hid1_bias'])

        # 2nd Hidden Layer State
        valid_hid2State = tf.sigmoid(tf.matmul(valid_hid1State, self.weights['hid1_to_hid2_wght']) + self.biases['hid2_bias'])

        # 3nd Hidden Layer State
        valid_hid3State = tf.sigmoid(tf.matmul(valid_hid2State, self.weights['hid2_to_hid3_wght']) + self.biases['hid3_bias'])

        # Output Layer State
        validOutState = tf.nn.softmax(tf.matmul(valid_hid3State, self.weights['hid3_to_out_wght']) + self.biases['out_bias'])

        return dict(
            validData = validData,
            validLabels = validLabels,
            validPred = validOutState
        )



In [6]:
class SessionExec(GlabalVariables):
    
    def __init__(self):
        super(SessionExec, self).__init__()
        
    def trainModel(self, trainDataIN, trainLabelsIN, sess):    
        numBatches = int(trainDataIN.shape[0]/self.batchSize)
        for numBatch in np.arange(numBatches):
            batchData = trainDataIN[numBatch*self.batchSize : (numBatch+1)*self.batchSize]
            batchLabels = trainLabelsIN[numBatch*self.batchSize : (numBatch+1)*self.batchSize]

            feed_dict = {self.trainGraphDict['trainData']: batchData,
                        self.trainGraphDict['trainLabels']: batchLabels
                        }

            _, loss, tpred = sess.run([self.trainGraphDict['optimizer'],
                                        self.trainGraphDict['lossCE'],
                                        self.trainGraphDict['trainPred']], feed_dict=feed_dict)
            
            if ((numBatch+1)%10 == 0) or ((numBatch+1) == numBatches): 
                tacc = accuracy(tpred, batchLabels)
                print ("Fold: " + str(self.foldNUM+1) + 
                       ", Epoch: " + str(self.epoch+1)+ 
                       ", Mini Batch: " + str(numBatch+1) + 
                       ", Loss= " + "{:.6f}".format(loss) + 
                       ", Training Accuracy= " + "{:.5f}".format(tacc))
    

    
    def validModel(self, validDataIN, validLabelsIN, sess):
#         print ('yyyyyytytytytytytytytytytytytytytytyty')
        feed_dict = {self.validGraphDict['validData']: validDataIN,
                     self.validGraphDict['validLabels']: validLabelsIN
                }

        vpred = sess.run(self.validGraphDict['validPred'], feed_dict=feed_dict)
        
        vacc = accuracy(vpred, validLabelsIN)
        print ("Fold: " + str(self.foldNUM+1) + 
                ", Epoch: " + str(self.epoch+1)+ 
                ", Validation Accuracy= " + "{:.5f}".format(vacc))
        
        return vpred 
        
        
    def execute(self):
        meanValidAcc = 0
        
        for foldNUM, (trainDataIN, trainLabelsIN, validDataIN, validLabelsIN, labelDict) in enumerate(genTrainValidFolds(featureDIR, oneHot=True)):   
            print ('')
            print ('##########################################################################################')

            print ('')
            print ('Validation Data and Labels shape: ', validDataIN.shape, validLabelsIN.shape)
            print ('Training Data and Labels shape: ', trainDataIN.shape, trainLabelsIN.shape)
            print ('The Label Dictionary is given as: ', labelDict)
            print ('')
            
            self.foldNUM = foldNUM

            reset_graph()

            # Create a object encapsulating the graph lineage
            objCNN = BuildNeuralNet(featureSize=trainDataIN.shape[1])
            self.trainGraphDict = objCNN.trainGraph()
            self.validGraphDict = objCNN.validGraph()
            
            with tf.Session() as sess:
                sess.run(tf.initialize_all_variables())

                for epoch in range(self.epochs): 
                    self.epoch = epoch
                    
                    # Train The model for Epochs
                    self.trainModel(trainDataIN, trainLabelsIN, sess)
                    print ('')

                    # Validate the model for Epochs
                    vpred = self.validModel(validDataIN, validLabelsIN, sess)
                    print ('')

                    if (epoch+1)%10 ==0:
                        validCM = confusionMatrix(validLabelsIN,vpred)
                        print ('Confusion Matrix CrossValid Set')
                        print (validCM)
                        print ('')
                    
            if foldNUM == 2:
                break

SessionExec().execute()   


##########################################################################################

Validation Data and Labels shape:  (1000, 1098) (1000, 2)
Training Data and Labels shape:  (9000, 1098) (9000, 2)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}

Fold: 1, Epoch: 1, Mini Batch: 10, Loss= 26857.056641, Training Accuracy= 58.60000
Fold: 1, Epoch: 1, Mini Batch: 18, Loss= 21474.130859, Training Accuracy= 63.60000

Fold: 1, Epoch: 1, Validation Accuracy= 56.70000

Fold: 1, Epoch: 2, Mini Batch: 10, Loss= 15337.061523, Training Accuracy= 69.00000
Fold: 1, Epoch: 2, Mini Batch: 18, Loss= 14767.106445, Training Accuracy= 71.00000

Fold: 1, Epoch: 2, Validation Accuracy= 57.00000

Fold: 1, Epoch: 3, Mini Batch: 10, Loss= 13508.179688, Training Accuracy= 75.40000
Fold: 1, Epoch: 3, Mini Batch: 18, Loss= 13239.704102, Training Accuracy= 74.60000

Fold: 1, Epoch: 3, Validation Accuracy= 57.50000

Fold: 1, Epoch: 4, Mini Batch: 10, Loss= 10227.8

In [7]:


# class SessionExec():
    
#     def __init__(self):
#         self.epochs = 30
        
#     def trainModel(self, trainDataIN, trainLabelsIN):
#         with tf.Session() as sess:
#             sess.run(tf.initialize_all_variables())

#             for epoch in range(self.epochs): 
#                 feed_dict = {self.trainGraphDict['trainData']: trainDataIN,
#                              self.trainGraphDict['trainLabels']: trainLabelsIN,
#                              self.trainGraphDict['keep_prob']: 0.7
#                         }

#                 if epoch == self.epochs-1:  # Capture the weights for the last iteration.
#                     print ('ENtering the last iteration.....', epoch)
#                     wLRND, bLRND, _, loss, tpred = sess.run([self.trainGraphDict['weightsLRND'],
#                                                 self.trainGraphDict['biasesLRND'],
#                                                 self.trainGraphDict['optimizer'],
#                                                 self.trainGraphDict['lossCE'],
#                                                 self.trainGraphDict['trainPred']], feed_dict=feed_dict)
#                 else:
#                     _, loss, tpred = sess.run([self.trainGraphDict['optimizer'], 
#                                                 self.trainGraphDict['lossCE'],
#                                                 self.trainGraphDict['trainPred']], feed_dict=feed_dict)

#                 if (epoch+1)%2 == 0 or epoch==self.epochs-1:
#                     # Evaluate training set
#                     tacc = accuracy(tpred, trainLabelsIN)
#                     print ("Fold: " + str(self.foldNUM+1) + ", Iter: " + str(epoch+1) + ", Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(tacc))
    
# #             print (wLRND)
# #             print ('')
# #             print (bLRND)
#             return tpred, tacc, wLRND, bLRND

                
#     # After all epoch are over with, test the model with the cross validation set    
#     def validateModel(self, validDataIN, validLabelsIN):
#         with tf.Session() as sess:
#             sess.run(tf.initialize_all_variables())
            
#             feed_dict = {self.validGraphDict['validData']: validDataIN,
#                      self.validGraphDict['validLabels']: validLabelsIN
#                 }

#             vpred = sess.run(self.validGraphDict['validPred'], feed_dict=feed_dict)
            
#             # Evaluate corss-validation set
#             vacc = accuracy(vpred, validLabelsIN)
#             print ("Fold: " + str(self.foldNUM+1) + ", Cross Validation Accuracy= " + "{:.5f}".format(vacc))

#             print ('')
#             print ('')
            
#             return vpred, vacc 
        
    
    
#     def sumWghtBias(self, cmnDictIN, newDictIN):
#         for param, val_aray in newDictIN.items():
#             cmnDictIN[param] = np.add(val_aray, cmnDictIN[param])    
#         return cmnDictIN
            
        
#     def execute(self, updWghtBias=False):
#         meanValidAcc = 0
#         self.wLRND = {}
#         self.bLRND = {}
#         for foldNUM, (trainDataIN, trainLabelsIN, validDataIN, validLabelsIN, labelDict) in enumerate(genTrainValidFolds(featureDIR, oneHot=True)):
#             print ('')
#             print ('Validation Data and Labels shape: ', validDataIN.shape, validLabelsIN.shape)
#             print ('Training Data and Labels shape: ', trainDataIN.shape, trainLabelsIN.shape)
#             print ('The Label Dictionary is given as: ', labelDict)
#             self.foldNUM = foldNUM
            
#             reset_graph()
            
#             # Build a class Object
#             objNNET = BuildNeuralNet()
            
#             # Build the training Graph
#             self.trainGraphDict = objNNET.trainNet(regularization=False)
#             tpred, tacc, wLRND, bLRND = self.trainModel(trainDataIN, trainLabelsIN)
            
            
#             reset_graph()
            
#             # Add all the weights and biases received from each training fold
#             if updWghtBias:
#                 if not (self.wLRND and self.bLRND):
#                     self.wLRND = wLRND
#                     self.bLRND = bLRND
#                 else:
#                     self.wLRND = self.sumWghtBias(self.wLRND, wLRND)
#                     self.bLRND = self.sumWghtBias(self.bLRND, bLRND)

                    
#             self.validGraphDict = objNNET.crossValid(wLRND, bLRND)
#             vpred, vacc = self.validateModel(validDataIN, validLabelsIN)
            
#             trainCM = confusionMatrix(trainLabelsIN,tpred)
#             validCM = confusionMatrix(validLabelsIN,vpred)

#             meanValidAcc += vacc
#             print ('Confusion Matrix Training Set')
#             print (trainCM)
#             print ('')
#             print ('Confusion Matrix CrossValid Set')
#             print (validCM)
            
# #             if foldNUM ==2:
# #                 break
        
#         # Test the cross validation accuracy for the nth-fold when the weights are averaged 
#         # Find the average of all the weights and biases from the training folds and take their average
#         if updWghtBias:
#             wLRND = {k: v/(foldNUM+1) for k, v in self.wLRND.items()} 
#             bLRND = {k: v/(foldNUM+1) for k, v in self.bLRND.items()} 
#             self.validGraphDict = objNNET.crossValid(wLRND, bLRND)
#             vpred, vacc = self.validateModel(validDataIN, validLabelsIN)
#             print ("Fold: " + str(self.foldNUM+1) + ", Aveaged Weight Cross Validation Accuracy= " + "{:.5f}".format(vacc))
        
#         print ('')
#         print ('The Mean crossValidation Accuracy is: ', meanValidAcc/(foldNUM+1))
            
            
# SessionExec().execute(updWghtBias=False)




# # Running i is : 0
# # Validation Data and Labels shape:  (1000, 162) (1000, 2)
# # Training Data and Labels shape:  (9000, 162) (9000, 2)
# # The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}
# # Fold: 1, Iter: 10, Loss= 1.603019, Training Accuracy= 0.79000
# # Fold: 1, Cross Validation Accuracy= 0.79200